In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import time
import csv
from torch import nn as nn
from torch.nn.functional import normalize
from PIL import Image
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset, dataloader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [141]:
data_path = r"C:\Users\ccm51\Documents\ECGR_4105\Final_Project\dataset_csv_file.csv"
league_numpy = np.loadtxt(data_path, dtype = np.float32, delimiter = ",", skiprows=1)
classNames = ['Btop', 'Bjungle', 'Bmid','Bbot','Bsupport','Rtop','Rjungle','Rmid','Rbot','Rsupport']
print(league_numpy.shape)

# league_train, league_test = train_test_split(league_numpy, train_size = 0.8, test_size = 0.2)

# league_train_results = league_train[:,-1]
# league_train = np.delete(league_train, league_train.shape[1] - 1, -1)
# league_test_results = league_test[:,-1]
# league_test = np.delete(league_test, league_test.shape[1] - 1, -1)

(4028, 11)


In [135]:
League = torch.from_numpy(league_numpy)
print(League)
League.shape, League.dtype

tensor([[ 95.,  79.,  31.,  ...,  27.,   3.,   1.],
        [118.,  79.,  65.,  ...,  47.,  66.,   0.],
        [ 68.,  82., 115.,  ..., 100.,  85.,   0.],
        ...,
        [103.,  56.,  83.,  ...,  15.,  74.,   1.],
        [ 16.,  28.,  88.,  ...,   7.,  87.,   0.],
        [122.,  63.,  88.,  ...,  27.,  85.,   0.]])


(torch.Size([4028, 11]), torch.float32)

In [136]:
data = League[:, :-1]
data, data.shape, data.dtype

(tensor([[ 95.,  79.,  31.,  ...,  17.,  27.,   3.],
         [118.,  79.,  65.,  ...,  17.,  47.,  66.],
         [ 68.,  82., 115.,  ...,  65., 100.,  85.],
         ...,
         [103.,  56.,  83.,  ..., 112.,  15.,  74.],
         [ 16.,  28.,  88.,  ..., 148.,   7.,  87.],
         [122.,  63.,  88.,  ...,  83.,  27.,  85.]]),
 torch.Size([4028, 10]),
 torch.float32)

In [145]:
target = League[:,-1].long()
target, target.shape, target.dtype

(tensor([1, 0, 0,  ..., 1, 0, 0]), torch.Size([4028]), torch.int64)

In [147]:
data_mean = torch.mean(data, dim=0).unsqueeze(-1)
data_std = torch.std(data, dim=0).unsqueeze(-1)
data_var = torch.var(data, dim=0).unsqueeze(-1)

print(data_mean, data_std)
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[68.1375],
        [58.5991],
        [80.5285],
        [55.8049],
        [75.1269],
        [67.3811],
        [61.1713],
        [78.9034],
        [55.8145],
        [75.1172]]) tensor([[40.9148],
        [27.8160],
        [45.7788],
        [40.5035],
        [37.6525],
        [41.8276],
        [28.8116],
        [46.1141],
        [40.1602],
        [36.6489]])


RuntimeError: The size of tensor a (4028) must match the size of tensor b (10) at non-singleton dimension 0

In [142]:
train_loader = torch.utils.data.DataLoader(dataset=data_normalized, batch_size= 64, shuffle = True)
model = nn.Sequential(nn.Linear(1,512),
                    nn.Tanh(),
                    nn.Linear(512,2),
                    nn.LogSoftmax(dim=0))
learning_rate = 1e-3
optimizer = optim.SGD(model.parameters(), lr= learning_rate)
loss_fn = nn.NLLLoss()
n_epochs = 3

In [146]:
for epoch in range(n_epochs + 1):
    loss_train = 0.0
    t1 = time.process_time()
    for imgs, labels, *_ in train_loader:
        batch_size = labels.shape[0]
        print(imgs.type)
        outputs = model(imgs.view(batch_size,-1))
        print(outputs.type)
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
t2 = time.process_time()
print(f"Epoch: %d, Loss: %f, Time: {t2-t1,3}" % (epoch,float(loss)))

<built-in method type of Tensor object at 0x000001C10ACFC400>
<built-in method type of Tensor object at 0x000001C10AC6F8B0>


RuntimeError: expected scalar type Long but found Float